## Basic Time Data Comparison

- 시계열 데이터 피처를 추가한 데이터 셋
- day, hour, minute, weekday

In [4]:
import pandas as pd

# 데이터 로드
train = pd.read_csv('./train.csv')

# yymm 컬럼을 날짜 형식으로 변환 (연도는 임의로 설정)
train['yymm'] = pd.to_datetime('2024' + train['yymm'], format='%Y%m%d %H:%M')

# day, hour, minute 컬럼 생성
train['day'] = train['yymm'].dt.day         # 일
train['hour'] = train['yymm'].dt.hour       # 시
train['minute'] = train['yymm'].dt.minute   # 분

# weekday 컬럼 생성
train['weekday'] = train['day'] % 7         # 요일 (0: 월요일, 1: 화요일, ..., 6: 일요일)

# yymm 컬럼 삭제
train.drop('yymm', axis=1, inplace=True)

# 결과 출력
train.head(10)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,Target,day,hour,minute,weekday
0,-5.327,12.250,-3.294,-7.855,-1.196,13.824,-10.249,-3.04,-5.170,8.077,...,-9.978,-2.689,-0.951,-3.873,0.471,44.521,1,0,0,1
1,-5.267,12.916,-3.220,-7.788,-1.196,14.424,-10.249,-3.04,-4.970,8.027,...,-10.038,-2.652,-1.018,-3.503,0.361,35.027,1,0,10,1
2,-5.127,13.583,-3.130,-7.658,-1.196,15.081,-10.359,-3.04,-4.830,7.977,...,-10.001,-2.652,-1.051,-3.436,0.361,13.920,1,0,20,1
3,-5.060,14.250,-3.130,-7.532,-1.196,14.961,-10.359,-3.04,-4.830,7.927,...,-10.028,-2.552,-1.111,-3.346,0.261,28.410,1,0,30,1
4,-4.967,14.916,-3.094,-7.462,-1.196,15.454,-10.359,-3.04,-4.970,7.877,...,-10.111,-2.619,-1.141,-3.346,0.261,1.647,1,0,40,1
5,-4.967,15.583,-3.020,-7.388,-1.196,15.284,-10.419,-3.04,-4.860,7.827,...,-10.111,-2.689,-1.208,-3.346,0.171,6.360,1,0,50,1
6,-4.827,16.250,-2.920,-7.288,-1.196,15.351,-10.449,-3.04,-4.933,7.777,...,-10.171,-2.762,-1.275,-3.346,0.171,34.535,1,1,0,1
7,-4.797,16.250,-2.920,-7.222,-1.196,14.188,-10.516,-3.04,-4.860,7.727,...,-10.478,-2.689,-1.341,-3.206,0.071,21.335,1,1,10,1
8,-4.737,16.250,-2.830,-7.188,-1.196,14.048,-10.659,-3.04,-4.933,7.677,...,-10.744,-2.689,-1.451,-3.073,0.004,34.687,1,1,20,1
9,-4.900,16.250,-2.890,-7.188,-1.196,14.014,-10.659,-3.04,-4.430,7.627,...,-10.941,-2.792,-1.551,-2.706,0.038,34.136,1,1,30,1


In [5]:
len(train.columns)

31

## Base Model MAE

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
columns = ['V2', 'V7', 'V11', 'V14', 'V20', 'V21', 'Target']
X = train.drop(columns=columns)    # Target을 제외한 모든 컬럼을 X로 지정
y = train['Target']                 # Target 컬럼을 y로 지정

# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# 각 모델에 대해 학습 및 5-fold 교차검증 수행
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=mae_scorer)
    print(f'{model_name}: {scores.mean()}')

Linear Regression: 12.626012274127707
Ridge: 12.62592037774775
Lasso: 12.585438135678952
ElasticNet: 12.584203595494085
KNN: 13.210953416820512


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)    # Target을 제외한 모든 컬럼을 X로 지정
y = train['Target']                 # Target 컬럼을 y로 지정

# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'SVR': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
    'Decision Tree': DecisionTreeRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# 각 모델에 대해 학습 및 5-fold 교차검증 수행
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=mae_scorer)
    print(f'{model_name}: {scores.mean()}')

Linear Regression: 12.637136750627773
Ridge: 12.637026399245347
Lasso: 12.585632581483413
ElasticNet: 12.593851660139743
SVR: 12.57996796157455
Gradient Boosting: 12.641440212271323
Random Forest: 13.260339159984616
XGBoost: 13.989391104426366
LightGBM: 13.271059712686853
Decision Tree: 17.052526956923078


## Feature Selection MAE

### Filter Method

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'SVR': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# SelectKBest로 K-최고 특성 선택
test = SelectKBest(score_func=f_regression, k=X.shape[1])
fit = test.fit(X, y)

# 선택된 특성들의 인덱스를 내림차순으로 정렬
sorted_columns = np.argsort(fit.scores_)[::-1]

# 각 모델에 대해 최적의 특성 선택
for model_name, model in models.items():

    # 최적의 특성을 찾기 위한 변수 초기화
    best_score = float('inf')
    best_features = []

    # 최적의 특성 선택
    for i in range(1, X.shape[1] + 1):
        # 선택된 feature들의 인덱스
        fs = sorted_columns[:i]

        # 선택된 feature만 선택 (Pandas DataFrame에서 iloc 사용)
        X_selected = X.iloc[:, fs]
        
        # 선택된 feature들의 이름
        selected_feature_names = X.columns[fs].tolist()
        
        # 교차 검증
        mae = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer).mean()

        # 가장 성능이 좋은 MAE 및 feature를 저장
        if mae < best_score:
            best_score = mae
            best_features = selected_feature_names
    
    # 결과 출력
    print(f'{model_name} best score: {best_score}, num_features: {len(best_features)}, best features: {best_features}')

Linear Regression best score: 12.526764152572579, num_features: 5, best features: ['V7', 'V17', 'V10', 'V4', 'V25']
Ridge best score: 12.526763926952748, num_features: 5, best features: ['V7', 'V17', 'V10', 'V4', 'V25']
Lasso best score: 12.535082495159886, num_features: 4, best features: ['V7', 'V17', 'V10', 'V4']
ElasticNet best score: 12.52996843930851, num_features: 4, best features: ['V7', 'V17', 'V10', 'V4']
SVR best score: 12.539948577916677, num_features: 1, best features: ['V7']
Gradient Boosting best score: 12.611499597536667, num_features: 12, best features: ['V7', 'V17', 'V10', 'V4', 'V25', 'V8', 'day', 'V23', 'weekday', 'V21', 'hour', 'V3']


### Forward Selection

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'SVR': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# 각 모델에 대해 최적의 특성 선택
for model_name, model in models.items():
    
    # 최적의 특성을 찾기 위한 변수 초기화
    best_score = float('inf')
    best_features = []

    # 최적의 특성 선택
    sfs = SequentialFeatureSelector(model, n_features_to_select=5, direction='forward')
    fit = sfs.fit(X, y)

    # 선택된 피처
    fs = X.columns[fit.support_].tolist()

    # 선택된 feature 데이터프레임 생성
    X_selected = X.iloc[:, fit.get_support()]

    # 교차 검증
    mae = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer).mean()

    # 결과 출력
    print(f'{model_name} best score: {mae}, best features: {fs}')


Linear Regression best score: 12.524648835270032, best features: ['V3', 'V10', 'V17', 'V25', 'V26']
Ridge best score: 12.524648903536, best features: ['V3', 'V10', 'V17', 'V25', 'V26']
Lasso best score: 12.534578335004287, best features: ['V1', 'V3', 'V5', 'V10', 'V17']
ElasticNet best score: 12.528527107451348, best features: ['V1', 'V8', 'V10', 'V17', 'V25']
SVR best score: 12.53474101802219, best features: ['V9', 'V16', 'V23', 'minute', 'weekday']
Gradient Boosting best score: 12.62350506568849, best features: ['V11', 'V21', 'day', 'minute', 'weekday']


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'SVR': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# 각 모델에 대해 최적의 특성 선택
for model_name, model in models.items():
    
    # 최적의 특성을 찾기 위한 변수 초기화
    best_score = float('inf')
    best_features = []

    # 최적의 특성 선택
    sfs = SequentialFeatureSelector(model, n_features_to_select=10, direction='forward')
    fit = sfs.fit(X, y)

    # 선택된 피처
    fs = X.columns[fit.support_].tolist()

    # 선택된 feature 데이터프레임 생성
    X_selected = X.iloc[:, fit.get_support()]

    # 교차 검증
    mae = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer).mean()

    # 결과 출력
    print(f'{model_name} best score: {mae}, best features: {fs}')


Linear Regression best score: 12.529737155314695, best features: ['V3', 'V5', 'V9', 'V10', 'V16', 'V17', 'V25', 'V26', 'hour', 'minute']
Ridge best score: 12.529734213165918, best features: ['V3', 'V5', 'V9', 'V10', 'V16', 'V17', 'V25', 'V26', 'hour', 'minute']
Lasso best score: 12.534578335004287, best features: ['V1', 'V3', 'V5', 'V8', 'V9', 'V10', 'V12', 'V13', 'V14', 'V17']
ElasticNet best score: 12.528527107451348, best features: ['V1', 'V3', 'V5', 'V8', 'V9', 'V10', 'V12', 'V13', 'V17', 'V25']
SVR best score: 12.528099716580932, best features: ['V1', 'V5', 'V8', 'V9', 'V13', 'V16', 'V23', 'V24', 'minute', 'weekday']
Gradient Boosting best score: 12.600911558356511, best features: ['V3', 'V8', 'V11', 'V16', 'V21', 'V25', 'day', 'hour', 'minute', 'weekday']


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'SVR': SVR(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# 각 모델에 대해 최적의 특성 선택
for model_name, model in models.items():
    
    # 최적의 특성을 찾기 위한 변수 초기화
    best_score = float('inf')
    best_features = []

    # 최적의 특성 선택
    sfs = SequentialFeatureSelector(model, n_features_to_select=15, direction='forward')
    fit = sfs.fit(X, y)

    # 선택된 피처
    fs = X.columns[fit.support_].tolist()

    # 선택된 feature 데이터프레임 생성
    X_selected = X.iloc[:, fit.get_support()]

    # 교차 검증
    mae = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer).mean()

    # 결과 출력
    print(f'{model_name} best score: {mae}, best features: {fs}')


Linear Regression best score: 12.537597835005288, best features: ['V1', 'V3', 'V5', 'V9', 'V10', 'V11', 'V12', 'V16', 'V17', 'V18', 'V23', 'V25', 'V26', 'hour', 'minute']
Ridge best score: 12.537595081397397, best features: ['V1', 'V3', 'V5', 'V9', 'V10', 'V11', 'V12', 'V16', 'V17', 'V18', 'V23', 'V25', 'V26', 'hour', 'minute']
Lasso best score: 12.534578335004287, best features: ['V1', 'V3', 'V5', 'V8', 'V9', 'V10', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20']
ElasticNet best score: 12.528527107451348, best features: ['V1', 'V3', 'V5', 'V8', 'V9', 'V10', 'V12', 'V13', 'V15', 'V16', 'V17', 'V18', 'V19', 'V24', 'V25']
SVR best score: 12.529908727892664, best features: ['V1', 'V5', 'V8', 'V9', 'V10', 'V12', 'V13', 'V15', 'V16', 'V19', 'V23', 'V24', 'V26', 'minute', 'weekday']
Gradient Boosting best score: 12.595385583743532, best features: ['V2', 'V3', 'V8', 'V11', 'V16', 'V18', 'V20', 'V21', 'V23', 'V24', 'V25', 'day', 'hour', 'minute', 'weekday']


### Backward Elimination

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# RFE를 사용하여 feature selection
for model_name, model in models.items():
    rfe = RFE(model, n_features_to_select=5)
    fit = rfe.fit(X, y)

    fs = X.columns[fit.support_].tolist()
    X_selected = X.iloc[:, fit.get_support()]

    # 선택된 feature로 cross-validation 수행
    score = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer)

    print(f'{model_name} score: {score.mean()}, selected features: {fs}')

Linear Regression score: 12.543496571231898, selected features: ['V5', 'V14', 'V18', 'V22', 'V24']
Ridge score: 12.543495934779815, selected features: ['V5', 'V14', 'V18', 'V22', 'V24']
Lasso score: 12.53489854284295, selected features: ['V2', 'V4', 'V7', 'V10', 'V17']
ElasticNet score: 12.530659655893988, selected features: ['V4', 'V7', 'V10', 'V17', 'V25']
Gradient Boosting score: 12.678629917717105, selected features: ['V3', 'V4', 'V7', 'V24', 'V25']


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# RFE를 사용하여 feature selection
for model_name, model in models.items():
    rfe = RFE(model, n_features_to_select=10)
    fit = rfe.fit(X, y)

    fs = X.columns[fit.support_].tolist()
    X_selected = X.iloc[:, fit.get_support()]

    # 선택된 feature로 cross-validation 수행
    score = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer)

    print(f'{model_name} score: {score.mean()}, selected features: {fs}')

Linear Regression score: 12.539533958015243, selected features: ['V2', 'V4', 'V5', 'V9', 'V14', 'V16', 'V18', 'V20', 'V22', 'V24']
Ridge score: 12.539504326049787, selected features: ['V2', 'V4', 'V5', 'V9', 'V14', 'V16', 'V18', 'V20', 'V22', 'V24']
Lasso score: 12.545574264133979, selected features: ['V2', 'V4', 'V7', 'V10', 'V11', 'V17', 'V21', 'day', 'hour', 'weekday']
ElasticNet score: 12.538953353710987, selected features: ['V2', 'V4', 'V7', 'V10', 'V11', 'V17', 'V21', 'V25', 'V26', 'hour']
Gradient Boosting score: 12.632705359150984, selected features: ['V1', 'V3', 'V4', 'V6', 'V7', 'V16', 'V18', 'V23', 'V24', 'V25']


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer, mean_absolute_error

# 데이터 분할
X = train.drop('Target', axis=1)  # Target 컬럼 제외
y = train['Target']  # Target 컬럼

# 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# MAE를 평가 기준으로 사용하기 위해 scorer 정의
mae_scorer = make_scorer(mean_absolute_error)

# RFE를 사용하여 feature selection
for model_name, model in models.items():
    rfe = RFE(model, n_features_to_select=15)
    fit = rfe.fit(X, y)

    fs = X.columns[fit.support_].tolist()
    X_selected = X.iloc[:, fit.get_support()]

    # 선택된 feature로 cross-validation 수행
    score = cross_val_score(model, X_selected, y, cv=5, scoring=mae_scorer)

    print(f'{model_name} score: {score.mean()}, selected features: {fs}')

Linear Regression score: 12.541660747658305, selected features: ['V1', 'V2', 'V4', 'V5', 'V9', 'V13', 'V14', 'V16', 'V18', 'V20', 'V22', 'V23', 'V24', 'V25', 'V26']
Ridge score: 12.54168302335242, selected features: ['V1', 'V2', 'V4', 'V5', 'V9', 'V13', 'V14', 'V16', 'V18', 'V20', 'V22', 'V23', 'V24', 'V25', 'V26']
Lasso score: 12.546104803276302, selected features: ['V2', 'V4', 'V7', 'V10', 'V11', 'V17', 'V21', 'V23', 'V24', 'V25', 'V26', 'day', 'hour', 'minute', 'weekday']
ElasticNet score: 12.544976842921471, selected features: ['V2', 'V4', 'V7', 'V10', 'V11', 'V17', 'V21', 'V23', 'V24', 'V25', 'V26', 'day', 'hour', 'minute', 'weekday']
Gradient Boosting score: 12.678090837367284, selected features: ['V1', 'V3', 'V4', 'V6', 'V7', 'V9', 'V15', 'V16', 'V17', 'V18', 'V20', 'V23', 'V24', 'V25', 'minute']
